In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [2]:
# Load
X = np.load('/home/filsave/malviso/X.npy', allow_pickle=True)
y = np.load('/home/filsave/malviso/y.npy', allow_pickle=True)

In [3]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
import numpy as np
from PIL import Image

def gabor_filter(ksize, sigma, theta, lambd, gamma, psi):
    filter_kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lambd, gamma, psi, cv2.CV_32F)
    return filter_kernel

def apply_gabor_filter(img, filter_kernel):
    filtered_img = cv2.filter2D(img, cv2.CV_8UC3, filter_kernel)
    return filtered_img

def extract_features(filtered_img):
    mean = np.mean(filtered_img)
    var = np.var(filtered_img)
    feature_vector = np.concatenate((np.array([mean]), np.array([var])))
    return feature_vector

def Gabor_filter_extraction(img):
    # Define the parameters for the 24 Gabor filters
    ksize = 5
    sigma_list = [1, 2, 3]
    theta_list = [0, np.pi/8, np.pi/4, 3*np.pi/8]
    lambd_list = [0.05, 0.25]
    gamma = 0.5
    psi = 0.0

    filtered_img_list = []
    for sigma in sigma_list:
        for theta in theta_list:
            for lambd in lambd_list:
                # Create the filter kernel
                filter_kernel = gabor_filter(ksize, sigma, theta, lambd, gamma, psi)
                # Apply the filter to the image
                filtered_img = apply_gabor_filter(img, filter_kernel)
                filtered_img_list.append(filtered_img)
    feature_vector = []
    for i, filtered_img in enumerate(filtered_img_list):
      filtered_img = cv2.normalize(filtered_img, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8UC1)
      features = extract_features(filtered_img)
      feature_vector.append(features)

    hist = cv2.calcHist([img], [0], None, [16], [0, 256])
    feature_vector.append(hist)
    return feature_vector


In [5]:
total_des_test = [Gabor_filter_extraction(img) for img in X_test] 

total_des_train = [Gabor_filter_extraction(img) for img in X_train] 

# reshape the filters
tot_train_features = []

for i in total_des_train:
  x = np.hstack(i[:24])
  z = i[-1].flatten()
  w = np.concatenate((x,z))
  tot_train_features.append(w)


tot_test_features = []

for i in total_des_test:
  x = np.hstack(i[:24])
  z = i[-1].flatten()
  w = np.concatenate((x,z))
  tot_test_features.append(w)

In [6]:
tot_test_features = np.array(tot_test_features)

tot_train_features = np.array(tot_train_features)

In [10]:
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(tot_train_features, y_train)

score = clf.score(tot_test_features, y_test)
print(" Test accuracy: ", score)

In [8]:
from sklearn.metrics import f1_score

clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(tot_train_features, y_train)

# predict the target values for the test data
y_pred = clf.predict(tot_test_features)

f1 = f1_score(y_test, y_pred, average='macro')
print(" f1_score accuracy: ", f1)

 f1_score accuracy:  0.9014195287150482
